In [1]:
# !pip install -q -r requirements.txt')
# !pip install -q stanza
from adjective_reading import *

In [2]:
def parse_keyword_sents(keyword, path_corpus=PATH_CORPUS, must_contain=MUST_CONTAIN, limit=None,**kwargs):
    for (url,sent) in iter_keyword_sents(keyword=keyword, path_corpus=path_corpus, must_contain=must_contain, limit=limit, **kwargs):
        if keyword and keyword.lower() not in sent.lower():
            continue
        yield (url, sent, get_nlp_doc(sent,id=url))

def iter_parsed_sents(limit=None,**kwargs):
    import stanza
    avail = len(STASH_NLP)
    total = limit if limit and limit < avail else avail
    for i, (key, val) in enumerate(STASH_NLP.items(), 1):
        if i > total:
            break

        if len(key)==3 and key[0]:
            url, sentstr = key
            sentdoc = stanza.Document.from_serialized(val)
            yield url, sentstr, sentdoc


NameError: name 'PATH_CORPUS' is not defined

In [8]:
for senturl,sentstr,sentdoc in parse_keyword_sents('reading', limit=1000):
    break
# senturl,sentstr,sentdoc

  0%|          | 2/71902 [00:00<04:08, 288.94it/s]


In [9]:
sentstr

'We are all our lifetime reading the copious sense of this first of forms.'

In [10]:

def get_parsed_sent_stats(sentdoc, keyword = None):
    tok_ld = sentdoc.to_dict()
    tok_id2obj = {}
    tok_id2d = {}
    tok_id2deprel = {}
    tok_id2head = {}
    tok_dd = {}

    for sent in sentdoc.sentences:
        for tok in sent.tokens:
            tok_d = tok.to_dict()[0]
            tok_id = tok_d['id']
            tok_id2d[tok_id] = tok_d
            tok_id2obj[tok_id] = tok

    out_ld = []
    for tok_id, tok_d in tok_id2d.items():
        tok_text = tok_d['text']
        if not keyword or tok_text.lower() == keyword.lower():
            tok_head_id = tok_d.get('head',0)
            tok_head_d = tok_id2d.get(tok_head_id,{})
            tok_id2head[tok_id] = tok_head_id
            ttext = tok_d.get("text","[text]")
            tlemma = tok_d.get("lemma",ttext)
            thead = tok_head_text = tok_head_d.get("text","[head]").lower()
            tdeprel = tok_id2deprel[tok_id] = tok_d.get('deprel',"[rel]")
            all_rels = [
                {
                    'id':w.id,
                    'deprel':w.deprel,
                    'head':w.head,
                }
                for w in sent.words
                if w.head == tok_id
            ]
            pprint(all_rels)
            
            token_ids_uppercase = [dx['id'] for dx in all_rels]
            sentstr2 = detokenize_and_uppercase(sentdoc, token_ids=token_ids_uppercase)

            for trel_d in all_rels:
                out_d = {
                    'source_id': tok_id,
                    'head_id': trel_d['id'],
                    'source': ttext,
                    'dep': trel_d['deprel'],
                    'head': trel_d['head'],
                    'sent': sentstr2
                }
            out_ld.append(out_d)
    return out_ld


In [55]:
def get_token_id2obj(sentdoc):
    tok_id2obj = {}
    for sent in sentdoc.sentences:
        for tok in sent.words:
            tok_id2obj[tok.id] = tok
    return tok_id2obj

# def get_token_span(sentdoc, tok_id_from, tok_id_to):
#     span = []
#     for sent in sentdoc.sentences:
#         for tok in sent.words:
#             if tok.id >= tok_id_from and tok.id <= tok_id_to:
#                 span.append(token)
#     return detokenize_and_uppercase(sentdoc, token_ids=span)

def get_token_info(sentdoc, tok_id):
    id2obj = get_token_id2obj(sentdoc)
    
    tok = id2obj[tok_id]
    tok_d = tok.to_dict()

    all_deps = []
    for id,tokx in id2obj.items():
        if tokx.head == tok_id:
            all_deps.append({'head_id':tokx.id,  'head_text':tokx.text, 'head_rel':tokx.deprel, 'head_type':'dep'})
        
    all_heads = []
    head_id = tok_d.get('head',0)
    if head_id and head_id in id2obj:
        head_tok = id2obj[head_id]
        head_tok_d = head_tok.to_dict()
        head_text = head_tok_d.get('text', '')
        head_deprel = head_tok_d.get('deprel', '')
        all_heads.append({'head_id':head_id, 'head_text':head_text, 'head_rel':tok_d.get('deprel', ''), 'head_type':'head'})
    
    all_rels = all_heads + all_deps

    out_ld = []
    for rel_d in all_rels:
        dephead = rel_d.get('head_type')
        tok_text = tok_d.get('text','')
        head_text = rel_d.get('head_text','')
        head_rel = rel_d.get('head_rel','')
        deprel = tok_d.get('deprel','')
        # if dephead == 'head':
            # rel_d['label'] = f"{tok_text} -- ({head_rel}) -- {head_text}"
        # else:
            # rel_d['label'] = f"{head_text} -- ({head_rel}) -- {tok_text}"
        rel_d['label'] = f"{head_rel}({head_text}, {tok_text})"
        out_d = {'id':tok_id,  **rel_d, 'text':tok_d.get('text', '')}
        out_ld.append(out_d)
    return [d for d in out_ld if d.get('head_rel')!='punct']
        
def get_tokens_info(sentdoc, keyword=None):
    out_ld = []
    for sent in sentdoc.sentences:
        for tok in sent.words:
            if keyword and tok.text.lower() != keyword.lower():
                continue
            out_ld.extend(get_token_info(sentdoc, tok.id))
    return out_ld



In [56]:
def iter_parsed_sents_stats(keyword, limit=None,**kwargs):
    for sent_id,(senturl,sentstr,sentdoc) in enumerate(parse_keyword_sents(keyword=keyword, limit=limit,**kwargs)):
        out_ld = get_tokens_info(sentdoc, keyword=keyword)
        for out_d in out_ld:
            sentstr2 = detokenize_and_uppercase(sentdoc, token_ids=[out_d['id'], out_d['head_id']])
            yield {'url':senturl, 'sent_id':sent_id, 'sent_str':sentstr, 'sent_repr':sentstr2, **out_d}

In [63]:
sentdoc = get_nlp_doc("Difficulty of reading is a good way to learn.")

In [64]:
get_tokens_info(sentdoc, keyword='reading')

[{'id': 3,
  'head_id': 1,
  'head_text': 'Difficulty',
  'head_rel': 'nmod',
  'head_type': 'head',
  'label': 'nmod(Difficulty, reading)',
  'text': 'reading'},
 {'id': 3,
  'head_id': 2,
  'head_text': 'of',
  'head_rel': 'case',
  'head_type': 'dep',
  'label': 'case(of, reading)',
  'text': 'reading'}]

In [49]:
odf = pd.DataFrame(iter_parsed_sents_stats('reading', limit=250))

  0%|          | 227/71902 [00:00<05:09, 231.65it/s]


In [50]:
odf2=odf.set_index(['url','sent_id','sent_str','id','head_id','label']).sort_index()
# odf2.drop(columns=['sent_repr','text',])
# odf.set_index(['text','head_rel','head_text']).sort_index()
# odf2.query('head_rel=="amod"')
odf2[odf2.head_type=="head"]

sent_repr  \
url                                        sent_id sent_str                                           id head_id label                                                                            
http://www.jstor.org/stable/10.1086/431560 215     The third, on Measure for Measure, is for me th... 23 15      appos(part, reading)         The third, on Measure for Measure, is for me t...   
                                           216     (Stanley Cavell’s reading of the play is highly... 5  7       det(existence, the)          (Stanley Cavell’s READING of THE play is highl...   
http://www.jstor.org/stable/10.1086/499175 161     That his- torical awareness is necessary for an... 33 31      obj(do, reading)             That his- torical awareness is necessary for a...   
http://www.jstor.org/stable/10.1086/511716 116     Not only do they riff on the ancient chestnut a... 27 28      nsubj(must, reading)         Not only do they riff on the ancient chestnut ...   
http://www.jstor.org/stable/10.1086/598693 24      Although my reading in Latin Christianity has i... 3  10      nsubj(led, reading)          Although my READING in Latin Christianity has ...   
...                                                                                                                                                                                         ...   
http://www.jstor.org/stable/516178         204     The second sentence does not make sense as it s... 28 25      advcl(attempt, reading)      The second sentence does not make sense as it ...   
                                           205     Then a wide- ranging chapter-the best in the bo... 22 19      nmod(response, reading)      Then a wide- ranging chapter-the best in the b...   
http://www.jstor.org/stable/518123         176     If they go too far in one direction they may an... 20 18      obj(making, reading)         If they go too far in one direction they may a...   
http://www.jstor.org/stable/518314         166     Indeed, just as the Edin- burgh Edition of the ... 32 26      obl(dismisses, reading)      Indeed, just as the Edin- burgh Edition of the...   
http://www.jstor.org/stable/518626         78      It should be recommended reading for any compar... 5  4       xcomp(recommended, reading)  It should be RECOMMENDED READING for any compa...   

                                                                                                                                                head_text  \
url                                        sent_id sent_str                                           id head_id label                                      
http://www.jstor.org/stable/10.1086/431560 215     The third, on Measure for Measure, is for me th... 23 15      appos(part, reading)                part   
                                           216     (Stanley Cavell’s reading of the play is highly... 5  7       det(existence, the)            existence   
http://www.jstor.org/stable/10.1086/499175 161     That his- torical awareness is necessary for an... 33 31      obj(do, reading)                      do   
http://www.jstor.org/stable/10.1086/511716 116     Not only do they riff on the ancient chestnut a... 27 28      nsubj(must, reading)                must   
http://www.jstor.org/stable/10.1086/598693 24      Although my reading in Latin Christianity has i... 3  10      nsubj(led, reading)                  led   
...                                                                                                                                                   ...   
http://www.jstor.org/stable/516178         204     The second sentence does not make sense as it s... 28 25      advcl(attempt, reading)          attempt   
                                           205     Then a wide- ranging chapter-the best in the bo... 22 19      nmod(response, reading)         response   
http://www.jstor.org/stable/518123         176     If they go too far in one

In [51]:
# odf.head_rel.value_counts()

In [52]:
triples = odf.label.str.lower().tolist()
# triples

In [53]:
from collections import Counter
counts = Counter(triples)

In [54]:
counts.most_common(25)

[('det(the, reading)', 31),
 ('det(a, reading)', 20),
 ('case(of, reading)', 13),
 ('mark(of, reading)', 11),
 ('cop(is, reading)', 9),
 ('det(this, reading)', 8),
 ('mark(in, reading)', 6),
 ('case(with, reading)', 6),
 ('cc(and, reading)', 6),
 ('case(to, reading)', 6),
 ('nmod:poss(my, reading)', 5),
 ('nmod:poss(his, reading)', 5),
 ('amod(original, reading)', 5),
 ('case(in, reading)', 4),
 ('compound(race, reading)', 4),
 ('case(by, reading)', 3),
 ('nmod(lines, reading)', 3),
 ('nsubj(this, reading)', 3),
 ('amod(correct, reading)', 3),
 ('conj(writing, reading)', 3),
 ('case(from, reading)', 3),
 ('nmod:poss(our, reading)', 3),
 ('nmod:poss(her, reading)', 3),
 ('case(for, reading)', 3),
 ('compound(proof, reading)', 3)]

In [ ]:
# odf2.query("head_rel=='amod'")